In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.callbacks as cb
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from functools import partial
from project import *

In [2]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64
MAX_LENGTH = 200

In [3]:
train_examples, test_examples, dataset_info = get_datasets()
encoder = dataset_info.features['text'].encoder
train_dataset, test_dataset = transform_datasets(train_examples, test_examples, 
                                                 encoder=encoder, 
                                                 batch_size=BATCH_SIZE,  
                                                 max_length=MAX_LENGTH, 
                                                 buffer_size=BUFFER_SIZE)

In [4]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8

input_vocab_size = encoder.vocab_size
dropout_rate = 0.1

In [5]:
loss_function = tf.keras.losses.BinaryCrossentropy(from_logits=True)
transformer = TransformerEncoderClassifier(num_layers, d_model, num_heads, dff, 
                                           input_vocab_size, 
                                           pe_input=input_vocab_size, 
                                           rate=dropout_rate)
transformer.compile(optimizer='adam', loss=loss_function, metrics=['accuracy'])

checkpoint = tf.train.latest_checkpoint("./checkpoints")
if checkpoint is not None:
    print("Loading previously trained model.")
    transformer.load_weights(checkpoint)

Loading previously trained model.


In [6]:
EPOCHS = 100

In [14]:
tb = cb.TensorBoard(histogram_freq=1, embeddings_freq=1)
csv = cb.CSVLogger('training.csv')
early = cb.EarlyStopping(monitor='val_accuracy', patience=5)
save = cb.ModelCheckpoint(filepath="checkpoints/train",
         monitor='val_accuracy',
         save_best_only=True,
         save_weights_only=True)

model_history = transformer.fit(train_dataset, 
                                validation_data=test_dataset,  
                                validation_freq=1,
                                shuffle=False,
                                callbacks=[early, save, tb, csv],
                                epochs=EPOCHS)

Epoch 1/100
134/134 [==============================]0.0244 - accuracy: 0.984 - 0s 136ms/step - loss: 0.0122 - accuracy: 0.992 - 0s 121ms/step - loss: 0.0083 - accuracy: 0.994 - 0s 113ms/step - loss: 0.0062 - accuracy: 0.996 - 1s 108ms/step - loss: 0.0050 - accuracy: 0.996 - 1s 105ms/step - loss: 0.0046 - accuracy: 0.997 - 1s 103ms/step - loss: 0.0047 - accuracy: 0.997 - 1s 101ms/step - loss: 0.0234 - accuracy: 0.996 - 1s 99ms/step - loss: 0.0357 - accuracy: 0.994 - 1s 98ms/step - loss: 0.0321 - accuracy: 0.99 - 1s 97ms/step - loss: 0.0292 - accuracy: 0.99 - 1s 96ms/step - loss: 0.0268 - accuracy: 0.99 - 1s 96ms/step - loss: 0.0382 - accuracy: 0.99 - 1s 95ms/step - loss: 0.0355 - accuracy: 0.99 - 1s 94ms/step - loss: 0.0332 - accuracy: 0.99 - 1s 94ms/step - loss: 0.0311 - accuracy: 0.99 - 2s 94ms/step - loss: 0.0293 - accuracy: 0.99 - 2s 93ms/step - loss: 0.0277 - accuracy: 0.99 - 2s 94ms/step - loss: 0.0262 - accuracy: 0.99 - 2s 93ms/step - loss: 0.0346 - accuracy: 0.99 - 2s 93ms/step 

134/134 [==============================] - ETA: 15s - loss: 0.9513 - accuracy: 0.921 - ETA: 13s - loss: 0.5345 - accuracy: 0.953 - ETA: 13s - loss: 0.3784 - accuracy: 0.963 - ETA: 12s - loss: 0.4255 - accuracy: 0.964 - ETA: 12s - loss: 0.4322 - accuracy: 0.965 - ETA: 12s - loss: 0.3601 - accuracy: 0.971 - ETA: 12s - loss: 0.3087 - accuracy: 0.975 - ETA: 11s - loss: 0.2701 - accuracy: 0.978 - ETA: 11s - loss: 0.3321 - accuracy: 0.979 - ETA: 11s - loss: 0.3205 - accuracy: 0.979 - ETA: 11s - loss: 0.4260 - accuracy: 0.977 - ETA: 11s - loss: 0.3905 - accuracy: 0.979 - ETA: 11s - loss: 0.4231 - accuracy: 0.979 - ETA: 11s - loss: 0.4080 - accuracy: 0.979 - ETA: 11s - loss: 0.3808 - accuracy: 0.981 - ETA: 10s - loss: 0.4112 - accuracy: 0.980 - ETA: 10s - loss: 0.4586 - accuracy: 0.976 - ETA: 10s - loss: 0.4365 - accuracy: 0.976 - ETA: 10s - loss: 0.4377 - accuracy: 0.975 - ETA: 10s - loss: 0.4569 - accuracy: 0.969 - ETA: 10s - loss: 0.4401 - accuracy: 0.969 - ETA: 10s - loss: 0.4540 - accurac

134/134 [==============================] - ETA: 18s - loss: 0.5461 - accuracy: 0.937 - ETA: 16s - loss: 0.5135 - accuracy: 0.929 - ETA: 15s - loss: 0.3880 - accuracy: 0.947 - ETA: 14s - loss: 0.2974 - accuracy: 0.957 - ETA: 14s - loss: 0.4209 - accuracy: 0.953 - ETA: 14s - loss: 0.6199 - accuracy: 0.942 - ETA: 14s - loss: 0.5543 - accuracy: 0.944 - ETA: 13s - loss: 0.4854 - accuracy: 0.951 - ETA: 13s - loss: 0.4703 - accuracy: 0.953 - ETA: 13s - loss: 0.5568 - accuracy: 0.953 - ETA: 13s - loss: 0.5335 - accuracy: 0.956 - ETA: 13s - loss: 0.5461 - accuracy: 0.957 - ETA: 13s - loss: 0.5129 - accuracy: 0.959 - ETA: 12s - loss: 0.4762 - accuracy: 0.962 - ETA: 12s - loss: 0.4445 - accuracy: 0.964 - ETA: 12s - loss: 0.4167 - accuracy: 0.966 - ETA: 12s - loss: 0.4031 - accuracy: 0.967 - ETA: 12s - loss: 0.4125 - accuracy: 0.968 - ETA: 12s - loss: 0.3950 - accuracy: 0.969 - ETA: 12s - loss: 0.4508 - accuracy: 0.968 - ETA: 12s - loss: 0.4441 - accuracy: 0.969 - ETA: 12s - loss: 0.4323 - accurac

134/134 [==============================] - ETA: 18s - loss: 0.0304 - accuracy: 0.984 - ETA: 16s - loss: 0.0476 - accuracy: 0.976 - ETA: 16s - loss: 0.0352 - accuracy: 0.984 - ETA: 15s - loss: 0.0371 - accuracy: 0.984 - ETA: 15s - loss: 0.0305 - accuracy: 0.987 - ETA: 14s - loss: 0.0254 - accuracy: 0.989 - ETA: 14s - loss: 0.0259 - accuracy: 0.988 - ETA: 14s - loss: 0.0228 - accuracy: 0.990 - ETA: 14s - loss: 0.0203 - accuracy: 0.991 - ETA: 14s - loss: 0.0186 - accuracy: 0.992 - ETA: 13s - loss: 0.0174 - accuracy: 0.992 - ETA: 13s - loss: 0.0160 - accuracy: 0.993 - ETA: 13s - loss: 0.0149 - accuracy: 0.994 - ETA: 13s - loss: 0.0139 - accuracy: 0.994 - ETA: 13s - loss: 0.0136 - accuracy: 0.993 - ETA: 13s - loss: 0.0128 - accuracy: 0.994 - ETA: 13s - loss: 0.0213 - accuracy: 0.993 - ETA: 13s - loss: 0.0201 - accuracy: 0.993 - ETA: 12s - loss: 0.0274 - accuracy: 0.992 - ETA: 12s - loss: 0.0296 - accuracy: 0.991 - ETA: 12s - loss: 0.0370 - accuracy: 0.990 - ETA: 12s - loss: 0.0378 - accurac

134/134 [==============================] - ETA: 18s - loss: 6.5005e-04 - accuracy: 1.000 - ETA: 16s - loss: 0.0382 - accuracy: 0.9922    - ETA: 15s - loss: 0.0255 - accuracy: 0.994 - ETA: 15s - loss: 0.0191 - accuracy: 0.996 - ETA: 14s - loss: 0.0175 - accuracy: 0.993 - ETA: 14s - loss: 0.0556 - accuracy: 0.989 - ETA: 14s - loss: 0.0477 - accuracy: 0.991 - ETA: 14s - loss: 0.0420 - accuracy: 0.992 - ETA: 14s - loss: 0.0545 - accuracy: 0.991 - ETA: 13s - loss: 0.0491 - accuracy: 0.992 - ETA: 13s - loss: 0.0526 - accuracy: 0.991 - ETA: 13s - loss: 0.0562 - accuracy: 0.989 - ETA: 13s - loss: 0.0520 - accuracy: 0.990 - ETA: 13s - loss: 0.0484 - accuracy: 0.991 - ETA: 13s - loss: 0.0452 - accuracy: 0.991 - ETA: 13s - loss: 0.0424 - accuracy: 0.992 - ETA: 13s - loss: 0.0399 - accuracy: 0.992 - ETA: 12s - loss: 0.0377 - accuracy: 0.993 - ETA: 12s - loss: 0.0401 - accuracy: 0.992 - ETA: 12s - loss: 0.0416 - accuracy: 0.992 - ETA: 12s - loss: 0.0396 - accuracy: 0.992 - ETA: 12s - loss: 0.0378 -

...

134/134 [==============================] - ETA: 17s - loss: 3.7563e-05 - accuracy: 1.000 - ETA: 15s - loss: 4.3096e-04 - accuracy: 1.000 - ETA: 14s - loss: 3.1687e-04 - accuracy: 1.000 - ETA: 14s - loss: 2.4057e-04 - accuracy: 1.000 - ETA: 14s - loss: 2.4110e-04 - accuracy: 1.000 - ETA: 14s - loss: 2.0126e-04 - accuracy: 1.000 - ETA: 13s - loss: 0.0083 - accuracy: 0.9978    - ETA: 13s - loss: 0.0073 - accuracy: 0.998 - ETA: 13s - loss: 0.0064 - accuracy: 0.998 - ETA: 13s - loss: 0.0059 - accuracy: 0.998 - ETA: 13s - loss: 0.0065 - accuracy: 0.998 - ETA: 13s - loss: 0.0060 - accuracy: 0.998 - ETA: 12s - loss: 0.0055 - accuracy: 0.998 - ETA: 12s - loss: 0.0051 - accuracy: 0.998 - ETA: 12s - loss: 0.0048 - accuracy: 0.999 - ETA: 12s - loss: 0.0045 - accuracy: 0.999 - ETA: 12s - loss: 0.0042 - accuracy: 0.999 - ETA: 12s - loss: 0.0040 - accuracy: 0.999 - ETA: 12s - loss: 0.0081 - accuracy: 0.998 - ETA: 12s - loss: 0.0079 - accuracy: 0.998 - ETA: 11s - loss: 0.0075 - accuracy: 0.998 - ETA: 

134/134 [==============================] - ETA: 18s - loss: 0.1513 - accuracy: 0.968 - ETA: 15s - loss: 0.0811 - accuracy: 0.984 - ETA: 15s - loss: 0.1188 - accuracy: 0.974 - ETA: 14s - loss: 0.0968 - accuracy: 0.976 - ETA: 14s - loss: 0.0775 - accuracy: 0.981 - ETA: 14s - loss: 0.0646 - accuracy: 0.984 - ETA: 13s - loss: 0.0553 - accuracy: 0.986 - ETA: 13s - loss: 0.0484 - accuracy: 0.988 - ETA: 13s - loss: 0.1036 - accuracy: 0.986 - ETA: 13s - loss: 0.1885 - accuracy: 0.984 - ETA: 13s - loss: 0.1713 - accuracy: 0.985 - ETA: 13s - loss: 0.1571 - accuracy: 0.987 - ETA: 12s - loss: 0.1450 - accuracy: 0.988 - ETA: 12s - loss: 0.1346 - accuracy: 0.988 - ETA: 12s - loss: 0.1257 - accuracy: 0.989 - ETA: 12s - loss: 0.1179 - accuracy: 0.990 - ETA: 12s - loss: 0.1585 - accuracy: 0.989 - ETA: 12s - loss: 0.2122 - accuracy: 0.987 - ETA: 12s - loss: 0.2144 - accuracy: 0.986 - ETA: 12s - loss: 0.2060 - accuracy: 0.985 - ETA: 11s - loss: 0.1962 - accuracy: 0.986 - ETA: 11s - loss: 0.1872 - accurac

134/134 [==============================] - ETA: 22s - loss: 1.1289e-09 - accuracy: 1.000 - ETA: 19s - loss: 7.6104e-09 - accuracy: 1.000 - ETA: 18s - loss: 4.1735e-07 - accuracy: 1.000 - ETA: 18s - loss: 5.1542e-07 - accuracy: 1.000 - ETA: 18s - loss: 4.5523e-07 - accuracy: 1.000 - ETA: 17s - loss: 4.0617e-07 - accuracy: 1.000 - ETA: 17s - loss: 5.2046e-05 - accuracy: 1.000 - ETA: 17s - loss: 0.0167 - accuracy: 0.9961    - ETA: 17s - loss: 0.0149 - accuracy: 0.996 - ETA: 16s - loss: 0.0134 - accuracy: 0.996 - ETA: 16s - loss: 0.0122 - accuracy: 0.997 - ETA: 16s - loss: 0.0112 - accuracy: 0.997 - ETA: 16s - loss: 0.0103 - accuracy: 0.997 - ETA: 16s - loss: 0.0096 - accuracy: 0.997 - ETA: 16s - loss: 0.0090 - accuracy: 0.997 - ETA: 16s - loss: 0.0084 - accuracy: 0.998 - ETA: 15s - loss: 0.0079 - accuracy: 0.998 - ETA: 15s - loss: 0.0075 - accuracy: 0.998 - ETA: 15s - loss: 0.0071 - accuracy: 0.998 - ETA: 15s - loss: 0.0067 - accuracy: 0.998 - ETA: 15s - loss: 0.0290 - accuracy: 0.996 - E

134/134 [==============================] - ETA: 22s - loss: 3.0261e-09 - accuracy: 1.000 - ETA: 19s - loss: 5.4294e-04 - accuracy: 1.000 - ETA: 18s - loss: 0.1209 - accuracy: 0.9948    - ETA: 17s - loss: 0.0907 - accuracy: 0.996 - ETA: 17s - loss: 0.0725 - accuracy: 0.996 - ETA: 17s - loss: 0.0713 - accuracy: 0.994 - ETA: 17s - loss: 0.0611 - accuracy: 0.995 - ETA: 17s - loss: 0.0535 - accuracy: 0.996 - ETA: 16s - loss: 0.0476 - accuracy: 0.996 - ETA: 16s - loss: 0.0433 - accuracy: 0.996 - ETA: 16s - loss: 0.0415 - accuracy: 0.995 - ETA: 16s - loss: 0.0381 - accuracy: 0.996 - ETA: 16s - loss: 0.0352 - accuracy: 0.996 - ETA: 16s - loss: 0.0326 - accuracy: 0.996 - ETA: 16s - loss: 0.0305 - accuracy: 0.996 - ETA: 15s - loss: 0.0286 - accuracy: 0.997 - ETA: 15s - loss: 0.0269 - accuracy: 0.997 - ETA: 15s - loss: 0.0254 - accuracy: 0.997 - ETA: 15s - loss: 0.0241 - accuracy: 0.997 - ETA: 15s - loss: 0.0229 - accuracy: 0.997 - ETA: 15s - loss: 0.0330 - accuracy: 0.997 - ETA: 15s - loss: 0.03

134/134 [==============================] - ETA: 17s - loss: 1.0904e-06 - accuracy: 1.000 - ETA: 15s - loss: 3.1266e-04 - accuracy: 1.000 - ETA: 14s - loss: 2.0938e-04 - accuracy: 1.000 - ETA: 14s - loss: 1.5884e-04 - accuracy: 1.000 - ETA: 14s - loss: 0.0358 - accuracy: 0.9969    - ETA: 13s - loss: 0.0299 - accuracy: 0.997 - ETA: 13s - loss: 0.0260 - accuracy: 0.997 - ETA: 13s - loss: 0.0228 - accuracy: 0.998 - ETA: 13s - loss: 0.0202 - accuracy: 0.998 - ETA: 13s - loss: 0.0183 - accuracy: 0.998 - ETA: 13s - loss: 0.0166 - accuracy: 0.998 - ETA: 12s - loss: 0.0152 - accuracy: 0.998 - ETA: 12s - loss: 0.0141 - accuracy: 0.998 - ETA: 12s - loss: 0.0137 - accuracy: 0.998 - ETA: 12s - loss: 0.0290 - accuracy: 0.997 - ETA: 12s - loss: 0.0308 - accuracy: 0.997 - ETA: 12s - loss: 0.0320 - accuracy: 0.996 - ETA: 12s - loss: 0.0302 - accuracy: 0.996 - ETA: 12s - loss: 0.0286 - accuracy: 0.996 - ETA: 12s - loss: 0.0294 - accuracy: 0.995 - ETA: 11s - loss: 0.0290 - accuracy: 0.994 - ETA: 11s - lo

In [ ]:
model_history.history

## Example of Evaluating a model

In [12]:
eval_loss, eval_acc = transformer.evaluate(test_dataset)

137/137 [==============================].9407 - accuracy: 0.82 - 0s 62ms/step - loss: 2.3711 - accuracy: 0.83 - 0s 56ms/step - loss: 2.5413 - accuracy: 0.82 - 0s 53ms/step - loss: 2.2473 - accuracy: 0.84 - 0s 51ms/step - loss: 2.2782 - accuracy: 0.84 - 0s 50ms/step - loss: 2.3829 - accuracy: 0.84 - 0s 49ms/step - loss: 2.4235 - accuracy: 0.84 - 0s 48ms/step - loss: 2.5407 - accuracy: 0.83 - 0s 47ms/step - loss: 2.3912 - accuracy: 0.84 - 0s 46ms/step - loss: 2.3815 - accuracy: 0.84 - 0s 45ms/step - loss: 2.3970 - accuracy: 0.83 - 1s 45ms/step - loss: 2.3889 - accuracy: 0.84 - 1s 45ms/step - loss: 2.4260 - accuracy: 0.84 - 1s 44ms/step - loss: 2.4920 - accuracy: 0.83 - 1s 44ms/step - loss: 2.4555 - accuracy: 0.84 - 1s 44ms/step - loss: 2.5827 - accuracy: 0.83 - 1s 44ms/step - loss: 2.5976 - accuracy: 0.82 - 1s 43ms/step - loss: 2.5069 - accuracy: 0.83 - 1s 43ms/step - loss: 2.4551 - accuracy: 0.83 - 1s 43ms/step - loss: 2.5631 - accuracy: 0.83 - 1s 43ms/step - loss: 2.5821 - accuracy: 0.

In [13]:
print(f'Loss: {eval_loss}, Acc: {eval_acc}')

Loss: 2.6485911242283176, Acc: 0.8304972648620605


### Examples of sentiment analysis

In [9]:
sent = partial(sentiment, encoder=encoder, transformer=transformer)

In [10]:
sent("This was not a very good movie.")

Input: This was not a very good movie.
Predicted sentiment: pos


In [11]:
sent('We loved the movie and definitely recommend it!')

Input: We loved the movie and definitely recommend it!
Predicted sentiment: pos
